# Digit Recogniser

The given task:
"In this competition, your goal is to correctly identify digits from a dataset of tens of thousands of handwritten images. We’ve curated a set of tutorial-style kernels which cover everything from regression to neural networks. We encourage you to experiment with different algorithms to learn first-hand what works well and how techniques compare."

The data:

    Training:
    42,000 images of digits
    Column 0: Number depicted by image
    Column 1-784: Pixel color (0 black, 1 white)
    

## 1. Preparing the data

Read files and extract information

In [7]:
%matplotlib inline
import pandas as pd

training = pd.read_csv("train.csv")

In [8]:
print(len(training))

42000
